### calculate the SA signal for each order

In [12]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import ascii, fits
from matplotlib.gridspec import GridSpec
from scipy import interpolate
from astropy.visualization import MinMaxInterval, AsinhStretch, HistEqStretch, ImageNormalize
from scipy.optimize import curve_fit
from scipy.stats import median_abs_deviation
%matplotlib inline

from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [13]:
# path to the directory containing the combined rectified and cal files
path = '/Users/jpw/Analysis/NIRSPEC/iSHELL/240107/'
path = '/Volumes/JPW_2TB/iSHELL/230630/'

In [14]:
# load atmospheric transmission file
hdu = fits.open('/Users/jpw/idl/Spextool/data/atran75000.fits')
tdata = hdu[0].data
atrans = interpolate.interp1d(tdata[0,:], tdata[1,:])
hdu.close()

In [15]:
j1 = 30           # starting point of first order -- figured out by hand
dj_AB = 121       # width of each order -- this should match the size of the number of rows in the order extension
dj_blank = 30     # gap between orders -- this is figured out by eye and assumed to be the same for all orders

In [16]:
# two gaussians with different peaks and offsets but same FWHM
# added an extra constant offset since some orders are not perfectly sky-subtracted
def gauss2(x, A1, x1, A2, x2, fwhm, C):
    sigma = fwhm / np.sqrt(8*np.log(2))
    y = A1 * np.exp(-0.5*((x-x1)/sigma)**2) + A2 * np.exp(-0.5*((x-x2)/sigma)**2) + C
    return y

In [17]:
def fit_median(order_flux, pix_nofit=0, plotfile=None):
    # stack an order in wavelength to measure the average slit profile and guide individual fits
    im_median = np.nanmedian(order_flux, axis=1)
    j = np.arange(im_median.size)
    p0 = [np.min(im_median), np.argmin(im_median), np.max(im_median), np.argmax(im_median), 2, 0]

    # don't fit within s_nofit of the edges
    fit_range = (j > pix_nofit) & (j < j[-1]-pix_nofit)

    # fit two gaussians to the data
    pfit, pcov = curve_fit(gauss2, j[fit_range], im_median[fit_range], p0)
    perr = np.sqrt(np.diag(pcov))

    if plotfile is not None:
        fig, ax = plt.subplots(figsize=(8, 4))
        ax.step(j, im_median)
        ax.plot(j[fit_range], gauss2(j[fit_range], *pfit))
        ax.set_xlabel(r"Row (pixels)", fontsize=14)
        ax.set_ylabel(r"Flux (Jy)", fontsize=14)
        fig.tight_layout()
        fig.savefig(plotfile, dpi=300)

    return pfit, perr

In [18]:
def remove_hot_pixels(order_flux, order_var, MAD_factor=20):
    # creating a mask on the variance image, putting to False all pixels that for each column are outside 5 sigma
    mask_var = np.full(order_var.shape, True)
    for i in range(order_var.shape[1]):
        column = np.nan_to_num(order_var[:,i])
        median = np.nanmedian(column)
        MAD = median_abs_deviation(column)    # Median Absolutet Deviation, much better than standard deviation since it's not influenced by outliers
        mask_column = np.logical_and(column>0, column<median+MAD_factor*MAD)    # 20 seems like a not too aggressive value
        mask_var[:,i] = mask_column

    # converting the mask from boolean to 0 and 1
    mask_var = np.array(mask_var, dtype=int)
    # convert zeros to NaNs
    mask_var = mask_var.astype(float)
    mask_var[mask_var==0] = np.nan
    
    order_flux_masked = order_flux*mask_var
    order_var_masked = order_var*mask_var
    
    return order_flux_masked, order_var_masked

$SA = \mu - \mu_\mathrm{MED}$ 

where 

$\mu = \frac{\sum_{i=0}^N y_i F_i}{\sum_{i=0}^N F_i}$ for each column

$\mu_\mathrm{MED}$ is the median value of $\mu$ within a horizontal window of M pixels


#### Uncertainties

$\Delta SA = \Delta \mu + \Delta \mu_\mathrm{MED}$

where

$\Delta \mu   = \left(\sum_{j=0}^N \left | \frac{y_j \left(\sum_{i=0}^N F_i \right) - \left(\sum_{i=0}^N y_i F_i \right)}{\left(\sum_{i=0}^N F_i \right)^2}  \right |  \Delta F_j\right) \frac{1}{\sqrt{N}} $ 

with $\Delta F_j\$ the error in the flux associated to each pixel

and 

$\Delta \mu_\mathrm{MED} = 1.253 \frac{\Delta \mu}{\sqrt{M}}$

In [19]:
def calculate_mu(order_flux, order_var, pix_nofit=0, nfwhm=1, C=1.,  verbose=False):
    ny, nx = order_flux.shape
    order_std = np.sqrt(order_var)
    
    # Fitting Gaussians to have the fwhm
    pfit, perr = fit_median(order_flux, pix_nofit=0) #, plotfile=path+source+'_fit_median.png')
    jneg =  pfit[1] + np.array([0, -0.5*nfwhm, 0.5*nfwhm]) * pfit[4]
    jpos =  pfit[3] + np.array([0, -0.5*nfwhm, 0.5*nfwhm]) * pfit[4]
    
    # Creating arrays
    mu_neg, mu_neg_err = np.zeros(nx), np.zeros(nx)   
    mu_pos, mu_pos_err = np.zeros(nx), np.zeros(nx)
    
    for i in range(nx):
        
        #############
        ### Calculating the mean of the vertical position for each column weighted by the pixel flux 
        #############
        
        # Negative
        flux_slice_neg = order_flux[int(np.around(jneg[1])): int(np.around(jneg[2]))+1, i]
        std_slice_neg = order_std[int(np.around(jneg[1])): int(np.around(jneg[2]))+1, i]
        yarr_neg = np.arange(int(np.around(jneg[1])), int(np.around(jneg[2]))+1)
        
        num = np.nansum(yarr_neg*flux_slice_neg)
        den = np.nansum(flux_slice_neg) 
        mu_neg[i] = num / den
        
        # Uncertainty
        der = (yarr_neg*den-num)/den**2
        mu_neg_err[i] = np.nansum(np.abs(der)*std_slice_neg) * 1/np.sqrt(yarr_neg.size)
    
    
        
        # Positive
        flux_slice_pos = order_flux[int(np.around(jpos[1])): int(np.around(jpos[2]))+1, i]
        std_slice_pos = order_std[int(np.around(jpos[1])): int(np.around(jpos[2]))+1, i]
        yarr_pos = np.arange(int(np.around(jpos[1])), int(np.around(jpos[2]))+1)
        
        num = np.nansum(yarr_pos*flux_slice_pos)
        den = np.nansum(flux_slice_pos)
        mu_pos[i] = num / den
        
        # Uncertainty
        der = (yarr_pos*den-num)/den**2
        mu_pos_err[i] = np.nansum(np.abs(der)*std_slice_pos) * 1/np.sqrt(yarr_pos.size)
    
    return mu_neg, mu_pos, mu_neg_err, mu_pos_err

In [20]:
def calculate_muMED(arr, arr_err, w):
    # Median filter 1D array arr over a window size w in pixels

    n = arr.size
    arr_filter, arr_fiter_err = np.zeros(n), np.zeros(n)
    
    # Define the window
    for i in range(n):
        
        # Left edge
        if i<int((w-1)/2):
            arr_window = arr[:i+1+int((w-1)/2)] 
        # Central pixels
        if int((w-1)/2)<= i < int(n - (w-1)/2):
            arr_window = arr[i-int((w-1)/2):i+1+int((w-1)/2)]
        # Right edge
        if i>=int(n - (w-1)/2):
            arr_window = arr[i-int((w-1)/2):] 
            
        arr_filter[i] = np.nanmedian(arr_window)
        arr_fiter_err[i] = 1.253 * arr_err[i]/np.sqrt(arr_window.size)
    
    return arr_filter, arr_fiter_err

In [21]:
def calculate_SA(mu, mu_err, muMED, muMED_err):
    nx = mu.size
    SA  = np.zeros((2, nx)) + np.nan

    SA[0] = mu - muMED
    SA[1] = mu_err + muMED_err
    
    return SA

In [22]:
# read in the source and wavecal from the manually edited sourcefile.txt
with open(path+'rectified/sourcelist.txt') as f:
    all_lines = f.read()
lines = all_lines.split('\n')

# find the breakpoints between sources
nbreak = []
for nline, line1 in enumerate(lines):
    if line1[0:4] == '----':
        nbreak.append(nline)
        
#select window size in pixels for median filter 
median_window_pixel_size = 300

# loop through the sources
print('-'*40)
for n1 in range(len(nbreak)-1):
    source = lines[nbreak[n1]+1]
    calfile = lines[nbreak[n1]+2]

    hdu1 = fits.open(path+'rectified/'+source+'_rectified.fits')
    flux = hdu1[0].data
    var = hdu1[1].data
    nslit, nwl = flux.shape

    hdu2 = fits.open(path+'cal/'+calfile)
    wc_hd = hdu2[0].header
    orders = wc_hd['ORDERS'].split(',')

    csvfile = open(path+'rectified/'+source+'_SA.csv', 'w')
    csvfile.write('wavelength,  off_neg, off_pos, err_neg, err_pos\n')

    # loop through the orders in reverse order so that the wavelength monotonically increases
    print(source)
    
    for n, order in enumerate(orders[::-1]):
        norder = len(orders) - n - 1
        j0 = j1 + (dj_AB + dj_blank) * norder

        wavecal = hdu2[3+norder].data
        wl_arr = wavecal[0, 0, 1:]
        print(f'Order = {order}, Min/Max wavelength = {wl_arr.min()}, {wl_arr.max()}')
        
        # Select and clean (remove hot pixels) orders
        order_flux = flux[j0:j0+dj_AB, :wl_arr.size]
        order_var = var[j0:j0+dj_AB, :wl_arr.size]
        order_flux_masked, order_var_masked = remove_hot_pixels(order_flux, order_var, MAD_factor=30)

        # Calculate mu and its error
        mu_neg, mu_pos, mu_neg_err, mu_pos_err = calculate_mu(order_flux_masked, order_var_masked, pix_nofit=0, nfwhm=3)
        
        # Calculate mu_MED and its error
        muMED_neg, muMED_neg_err = calculate_muMED(mu_neg, mu_neg_err, median_window_pixel_size)
        muMED_pos, muMED_pos_err = calculate_muMED(mu_pos, mu_pos_err, median_window_pixel_size)
        
        # Calculate SA and its error
        SA_neg = calculate_SA(mu_neg, mu_neg_err, muMED_neg, muMED_neg_err)
        SA_pos = calculate_SA(mu_pos, mu_pos_err, muMED_pos, muMED_pos_err)
        
        flag = atrans(wl_arr) < 0.3
        SA_neg[:, flag] = np.nan
        SA_pos[:, flag] = np.nan

        for i in range(wl_arr.size):
            csvfile.write(f'{wl_arr[i]:11.9f}, {SA_neg[0, i]:7.4f}, {SA_pos[0, i]:7.4f}, {SA_neg[1, i]:7.4f}, {SA_pos[1, i]:7.4f}\n')

    print('-'*40)
    hdu1.close()
    hdu2.close()
    csvfile.close()

----------------------------------------
Elias24_PA136
Order = 114, Min/Max wavelength = 4.509217059629072, 4.547327670478346
Order = 113, Min/Max wavelength = 4.548980656038084, 4.587426797393182
Order = 112, Min/Max wavelength = 4.589453823346046, 4.628245758975768
Order = 111, Min/Max wavelength = 4.630655739154617, 4.669804010227063
Order = 110, Min/Max wavelength = 4.672606278433143, 4.712121713602966
Order = 109, Min/Max wavelength = 4.715326045508029, 4.7552197714664
Order = 108, Min/Max wavelength = 4.758836407829275, 4.7991198603422855
Order = 107, Min/Max wavelength = 4.803159531630485, 4.8438444670933505
Order = 106, Min/Max wavelength = 4.848318419607356, 4.8894169271436345
Order = 105, Min/Max wavelength = 4.894336950749226, 4.935861464886195
Order = 104, Min/Max wavelength = 4.941239922468601, 4.98320323642202
Order = 103, Min/Max wavelength = 4.989053095184825, 5.031468374788592
Order = 102, Min/Max wavelength = 5.037803239530333, 5.080684037848972
Order = 101, Min/Max w